In [1]:
import os
import numpy as np
import yfinance as yf
from flask import Flask, jsonify, request
from binance.client import Client
from dotenv import load_dotenv
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

2025-06-06 14:04:06.229507: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
api_key = "dPhWMXJcC98Oelgl6KbaGgiL5T5PL0zgrEESP8g0oq656u3z6aOCrOkMz0SQpgWT"
api_secret = "JZEk6FSNdEMnTv8r7N4jcmn0JllUdA7D4yVtIakAV4Myy0qyciy5Q2zO8yLFPami"
symbol = "BTCUSDT"
LOOKBACK = 60
client = Client(api_key, api_secret)

In [5]:
ticker = symbol.replace("USDT", "-USD")
df = yf.download(ticker, period="1y", interval="1d")[['Close']]
df.head()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Price,Close
Ticker,BTC-USD
Date,
2024-06-06,70757.164062
2024-06-07,69342.585938
2024-06-08,69305.773438
2024-06-09,69647.992188
2024-06-10,69512.281250


In [11]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)

X, y = [], []
for i in range(LOOKBACK, len(scaled_data)):
    X.append(scaled_data[i-LOOKBACK:i, 0])
    y.append(scaled_data[i, 0])
X, y = np.array(X), np.array(y)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X.shape[1], 1)),
    LSTM(50),
    Dense(1)
])
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X, y, epochs=5, batch_size=32, verbose=0)

/Users/marco.zarate/repos/bot_predictor/.venv/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [19]:
def generate_signal(current, predicted, threshold=0.005):
    change = (predicted - current) / current
    if change > threshold:
        return "BUY"
    elif change < -threshold:
        return "SELL"
    else:
        return "HOLD"

In [24]:
data = scaled_data
price_info = client.get_symbol_ticker(symbol=symbol)
current_price = float(price_info["price"])
print(current_price)
last_60 = data[-LOOKBACK:]
if last_60.shape != (LOOKBACK, 1):
    last_60 = last_60.reshape((LOOKBACK, 1))
# Ensure last_60 is shaped (LOOKBACK, 1)
last_60 = data[-LOOKBACK:]
if last_60.shape != (LOOKBACK, 1):
    last_60 = last_60.reshape((LOOKBACK, 1))
X_input = np.reshape(last_60, (1, LOOKBACK, 1))
predicted_scaled = model.predict(X_input)
# Ensure predicted_scaled is 2D for inverse_transform
if predicted_scaled.ndim == 1:
    predicted_scaled = predicted_scaled.reshape(-1, 1)
predicted_price = scaler.inverse_transform(predicted_scaled)[0][0]

signal = generate_signal(current_price, predicted_price)

print("symbol",symbol)
print("current_price", current_price)
print("predicted_price", predicted_price)
print("signal", signal)

104678.11
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step
symbol BTCUSDT
current_price 104678.11
predicted_price 106176.71
signal BUY


In [29]:
jsonify({
    "symbol": symbol,
    "current_price": f"{current_price}",
    "predicted_price": f"{predicted_price}",
    "signal": signal
})

RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
the current application. To solve this, set up an application context
with app.app_context(). See the documentation for more information.